In [1]:
!pip install langchain-google-genai
!pip install pandas openpyxl
!pip install sqlite3
!pip install langchain
!pip install -U langchain-community
!pip install sqlalchemy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.0/384.0 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.2/140.2 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 11.6 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
ERROR: Could not find a version that satisfies the requirement sqlite3 (from versions: none)
ERROR: No matching distribution found for sqlite3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.6/990.6 kB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 81.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 4.4 MB/s eta 0:00:00


In [2]:
import os
import pandas as pd
import sqlite3
from IPython.display import display, Markdown
import textwrap
from langchain_google_genai import ChatGoogleGenerativeAI

In [3]:
# Set up the environment variable for the Google API key
os.environ["GOOGLE_API_KEY"] = "# Replace with your actual API key"  

# Initialize the LLM
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

In [4]:
def write_to_sqlite(df, table_name):
    sqlite_db_path = 'example.db'
    conn = sqlite3.connect(sqlite_db_path)
    df.to_sql(table_name, conn, if_exists='replace', index=False)
    conn.commit()
    conn.close()
    print(f"Data has been successfully written to the {table_name} table in the SQLite database.")

In [5]:
# Read the Excel files and display their contents to verify the 'id' column
employees_excel_file_path = 'employees.xlsx'
employees_df = pd.read_excel(employees_excel_file_path)
print("Employees DataFrame from Excel:")
print(employees_df.head())

salaries_excel_file_path = 'salaries.xlsx'
salaries_df = pd.read_excel(salaries_excel_file_path)
print("Salaries DataFrame from Excel:")
print(salaries_df.head())

Employees DataFrame from Excel:
   ID     Name  Age
0   1    Alice   30
1   2      Bob   25
2   3  Charlie   35
3   4    Diana   28
4   5   Edward   40
Salaries DataFrame from Excel:
   ID   Department   Salary
0   1  Engineering    70000
1   2    Marketing    60000
2   3  Engineering    80000
3   4            HR   50000
4   5  Engineering    90000


In [6]:
# Write to SQLite
write_to_sqlite(employees_df, "employees")
write_to_sqlite(salaries_df, "salaries")

def read_from_sqlite(table_name):
    sqlite_db_path = 'example.db'
    try:
        conn = sqlite3.connect(sqlite_db_path)
        query = f"SELECT * FROM {table_name}"
        df = pd.read_sql_query(query, conn)
        conn.close()
        return df
    except Exception as e:
        print(f"Error reading from SQLite table {table_name}: {e}")
        return pd.DataFrame()

Data has been successfully written to the employees table in the SQLite database.
Data has been successfully written to the salaries table in the SQLite database.


In [7]:
# Read tables from SQLite into DataFrames
employees_df = read_from_sqlite("employees")
salaries_df = read_from_sqlite("salaries")

print("Employees DataFrame from SQLite:")
print(employees_df.head())

print("\nSalaries DataFrame from SQLite:")
print(salaries_df.head())

Employees DataFrame from SQLite:
   ID     Name  Age
0   1    Alice   30
1   2      Bob   25
2   3  Charlie   35
3   4    Diana   28
4   5   Edward   40

Salaries DataFrame from SQLite:
   ID   Department   Salary
0   1  Engineering    70000
1   2    Marketing    60000
2   3  Engineering    80000
3   4            HR   50000
4   5  Engineering    90000


In [8]:
from IPython.display import display, Markdown
import textwrap
from langchain_google_genai import ChatGoogleGenerativeAI

# Function to format text as markdown
def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# Function to query the tables using Gemini AI
def query_tables(query):
    response = llm.invoke(query)
    return response

In [9]:
# Function to interact with tables and process the query
def interact_with_tables(query):
    if employees_df.empty or salaries_df.empty:
        return "One or both DataFrames are empty. Cannot perform merge."

    # Check for the 'id' column
    if 'ID' not in employees_df.columns or 'ID' not in salaries_df.columns:
        return "'id' column is missing from one or both DataFrames."

    # Combine data from both DataFrames
    combined_table = pd.merge(employees_df, salaries_df, on='ID')

    # Prepare the data as a string for the model to process
    data_str = combined_table.to_string(index=False)

    # Create a complete query including the data
    complete_query = f"{query}\n\nData:\n{data_str}"

    # Use AI to process the query
    response = query_tables(complete_query)
    return response

In [10]:
# Example usage
query = "List the names of employees who are Engineers and more than the age of 30."
result = interact_with_tables(query)
display(to_markdown(result.content))

> Based on the data provided, the employees who are Engineers and over 30 years old are:
> 
> * **Charlie** (Age: 35)
> * **Edward** (Age: 40) 


In [11]:
employees_df.columns

Index(['ID', 'Name', 'Age'], dtype='object')

In [12]:
salaries_df.columns

Index(['ID', 'Department ', 'Salary'], dtype='object')